# KNN & Naive Bayes

In [129]:
import pandas as pd
import numpy as np

In [130]:
data = pd.read_csv('Heart.csv')
data

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,299,45,1,typical,110,264,0,0,132,0,1.2,2,0.0,reversable,Yes
299,300,68,1,asymptomatic,144,193,1,0,141,0,3.4,2,2.0,reversable,Yes
300,301,57,1,asymptomatic,130,131,0,0,115,1,1.2,2,1.0,reversable,Yes
301,302,57,0,nontypical,130,236,0,2,174,0,0.0,2,1.0,normal,Yes


In [131]:
data.head()

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [132]:
data.tail()

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
298,299,45,1,typical,110,264,0,0,132,0,1.2,2,0.0,reversable,Yes
299,300,68,1,asymptomatic,144,193,1,0,141,0,3.4,2,2.0,reversable,Yes
300,301,57,1,asymptomatic,130,131,0,0,115,1,1.2,2,1.0,reversable,Yes
301,302,57,0,nontypical,130,236,0,2,174,0,0.0,2,1.0,normal,Yes
302,303,38,1,nonanginal,138,175,0,0,173,0,0.0,1,NaN,normal,No


In [133]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['ChestPain'] = le.fit_transform(data['ChestPain'].astype(str))
data['Thal'] = le.fit_transform(data['Thal'].astype(str))
data['AHD'] = le.fit_transform(data['AHD'].astype(str))
data.head()

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,3,145,233,1,2,150,0,2.3,3,0.0,0,0
1,2,67,1,0,160,286,0,2,108,1,1.5,2,3.0,2,1
2,3,67,1,0,120,229,0,2,129,1,2.6,2,2.0,3,1
3,4,37,1,1,130,250,0,0,187,0,3.5,3,0.0,2,0
4,5,41,0,2,130,204,0,2,172,0,1.4,1,0.0,2,0


In [134]:
zeroes_not_accepted = ['ChestPain','Fbs','RestECG','ExAng','Ca','Thal']
for column in zeroes_not_accepted:
    data[column] = data[column].replace(0,np.NaN)
    mean = int(data[column].mean())
    data[column] = data[column].replace(np.NaN,mean)

In [135]:
data.head()

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,3.0,145,233,1.0,2.0,150,1.0,2.3,3,1.0,2.0,0
1,2,67,1,1.0,160,286,1.0,2.0,108,1.0,1.5,2,3.0,2.0,1
2,3,67,1,1.0,120,229,1.0,2.0,129,1.0,2.6,2,2.0,3.0,1
3,4,37,1,1.0,130,250,1.0,1.0,187,1.0,3.5,3,1.0,2.0,0
4,5,41,0,2.0,130,204,1.0,2.0,172,1.0,1.4,1,1.0,2.0,0


In [136]:
#x = data[['Age','Sex','ChestPain','RestBP','Chol','Fbs','RestECG','MaxHR','ExAng','Oldpeak','Slope','Ca','Thal']].values
x = data.drop(['Unnamed: 0','AHD'],axis=1).values
y = data['AHD'].values
x

array([[63.,  1.,  3., ...,  3.,  1.,  2.],
       [67.,  1.,  1., ...,  2.,  3.,  2.],
       [67.,  1.,  1., ...,  2.,  2.,  3.],
       ...,
       [57.,  1.,  1., ...,  2.,  1.,  3.],
       [57.,  0.,  2., ...,  2.,  1.,  2.],
       [38.,  1.,  1., ...,  1.,  1.,  2.]])

In [137]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = 0.3)

In [138]:
y_test

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0])

In [139]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

# KNN

In [140]:
#For Knn n_neighbors
import math
math.sqrt(len(y_test))

9.539392014169456

In [141]:
#For KNN
from sklearn.neighbors import KNeighborsClassifier
cls = KNeighborsClassifier(n_neighbors=9,p=2,metric='euclidean')
cls.fit(x_train,y_train)
#n_neighbors = 11 bcoz Y_test=12 and its even so 12-1=11
#p=2 because we want to determine whether the patient is suffering from diabetes or not

KNeighborsClassifier(metric='euclidean', n_neighbors=9)

In [142]:
#For KNN
ky_pred = cls.predict(x_test)
ky_pred

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0])

In [143]:
#ConfusionMatrix for KNN
from sklearn.metrics import confusion_matrix
KNNcm = confusion_matrix(y_test,ky_pred)
KNNcm

array([[43,  7],
       [13, 28]])

In [144]:
#Accuracy score for KNN
from sklearn.metrics import accuracy_score
KNNacc = accuracy_score(y_test,ky_pred) * 100
KNNacc

78.02197802197803

# Naive Bayes

In [145]:
#For Naive Bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train,y_train)

GaussianNB()

In [146]:
#For Naive Bayes
ny_pred = model.predict(x_test)
ny_pred

array([1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0])

In [147]:
#ConfusionMatric for NBC
from sklearn.metrics import confusion_matrix
NBcm = confusion_matrix(y_test,ny_pred)
NBcm

array([[43,  7],
       [10, 31]])

In [148]:
#Accuracy score for NB
from sklearn.metrics import accuracy_score
NBacc = accuracy_score(y_test,ny_pred) * 100
NBacc

81.31868131868131